In [19]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize
!pip install folium

     |████████████████████████████████| 93 kB 3.2 MB/s  eta 0:00:01


## Assignment 1
## Reading file from the wiki pedia 
##   1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
##    2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
## More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
##    3.If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
##    4. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
##    5. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [26]:
table_CAN = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df_TN = table_CAN[0]
df_TN['Borough'].replace("Not assigned", np.nan, inplace = True)
df_TN.dropna(subset=["Borough"], axis=0, inplace=True)
# reset index, because we droped two rows
df_TN.reset_index(drop=True, inplace=True)
df_TN['Neighbourhood'].replace("Not assigned", df_TN['Borough'], inplace = True)
df_TN

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [27]:
df_TN.shape

(103, 3)

## Assignment 2 Add geospatial data to the data frame

In [28]:
df_TN_sortedby_postalcode = df_TN.sort_values("Postal Code")
df_TN_sortedby_postalcode
filename = 'http://cocl.us/Geospatial_data'
df = pd.read_csv(filename)
df.sort_values("Postal Code")
df_lat = pd.merge(df_TN_sortedby_postalcode, df, on='Postal Code')
df_lat

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


## Assignment 3

In [29]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [33]:
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
 # g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  #lat_lng_coords = g.latlng

latitude = 43.6532
longitude = 79.3832

In [34]:


# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_lat['Latitude'], df_lat['Longitude'], df_lat['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map